In [1]:
import itertools

In [2]:
# 1. Need a 6 or 7 week scheduled where you never play with the same
#    pods and where you only play against opposing pods no more than 3 times.

# 2. The top 12 pods don't mix with the bottom 12 pods (middle ranked pods have
#    free range). Maybe 8 pods each side if there are 36 pods instead of 42.

# 3. Games should be relatively balanced/competitive. Each player tanks their
#    skill level giving each pod a team ranking. Those rankings are trusted to help 
#    keep it balanced. E.g., the #1, 9, and 15 team would be 25 points. 
#    So they would play pods who add up close to 25 with some level of deviation allowed.

# 4. Ideally, this program would allow for a spot to input who won and lost to help us
#    keep track of records (but not necessary).

# F: You should be able to input a team's self-power ranking, number of teams, and 
#    number of weeks and output a balanced schedule with little repetitiveness.

# POD_COUNT = 48
# TEAMS_PER_POD = 3
# POD_PARTNERS_PER_WEEK = 2
# GAMES_PER_WEEK = 2
# SCHEDULE_WEEKS = 6
# TOP_BOTTOM_POD_COUNT = 8


In [3]:
class Pod:
    
    is_bottom = False
    is_top = False
    
    def __init__(self, name, rank):
        self.name = name
        self.rank = rank
        self.played_with = {}
        self.played_against = {}
        
    def play_with(self, pod):
        if pod in self.played_with:
            self.played_with[pod]['count'] += 1
            # self.played_with (things like week, etc. could go here)
        else:
            self.played_with[pod] = {'count': 0}        
        
    def play_against(self, pod):
        if pod in self.played_against:
            self.played_against[pod]['count'] += 1
        else:
            self.played_against[pod] = {'count': 1}
            
    def __str__(self):
        return str(self.name)
        
        
class Team:
    
    def __init__(self, pods):
        self.pods = pods
        self.score = sum(pod.rank for pod in pods)
        
    def __str__(self):
        return str([pod.name for pod in self.pods])
    
    def update_teams(self):
        for comb in itertools.combinations(self.pods, 2):
            comb[0].play_with(comb[1])
            comb[1].play_with(comb[0])
            
    def get_mean_played_against_team(self, opponent_team):
        """Returns the combined mean times each pod on a team played against a pod on another team"""
        played_running_total = 0
        for p1 in self.pods:
            for p2 in opponent_team.pods:
                if p2 in p1.played_against:
                    played_running_total += p1.played_against[p2].get('count', 0)
        mean_played_against = played_running_total / (len(self.pods) + len(opponent_team.pods))
        return mean_played_against
        
        
class Season:
    """
    - Each week, 3 teams combine together (and should only play with each other 1 week).
    - 2 games per week (so you interact with 8 other teams each week; 2 with and 6 against)
    - Ideally, shouldn't play against the same team more then 2x. 
    - Algorithm should work for over 6 weeks, while keeping teams balanced 
      (e.g. if I combine the 1st, 8th, and 12th best team (total 21), then their 
       competition should be close to them such as 2nd, 7th, and 11th (total 20)
      ). 
    """
    
    POD_COUNT = 48
    PODS_PER_TEAM = 3
    POD_PARTNERS_PER_WEEK = 2
    GAMES_PER_DAY = 2
    SCHEDULE_WEEKS = 6
    TOP_BOTTOM_POD_COUNT = 12
    MU = 50
    
    def __init__(self, pods):
        self.pods = pods
        self.sorted_pods = self.sort_pods_by_rank()
        self.valid_teams = self.generate_valid_teams()
        self.schedule = {}
#         self.valid_teams, self.valid_bottoms, self.valid_tops = self.generate_potential_teams()
#         self.teams = {}
        
    def sort_pods_by_rank(self):
        """Sort pods by their self-reported rank and record top and bottom flags"""
        sorted_pods = sorted(self.pods, key=lambda x: x.rank, reverse=False)
        for pod in sorted_pods[:self.TOP_BOTTOM_POD_COUNT]:
            pod.is_bottom = True
        for pod in sorted_pods[-self.TOP_BOTTOM_POD_COUNT:]:
            pod.is_top = True
        return sorted_pods
        
#     def generate_potential_teams(self):
#         """
#         Generate all the possible team pairings, respecting the rules of top
#         and bottom pairing restrictions
#         """        
#         valid_teams = []
#         valid_bottoms = []
#         valid_tops = []
#         all_teams = itertools.combinations(self.pods, 3)
#         for team in all_teams:
#             bottom = max(pod.is_bottom for pod in team)
#             top = max(pod.is_top for pod in team)
#             if not(bottom and top):
#                 t = Team(pods=team)
#                 valid_teams.append(t)
#             if not bottom:
#                 valid_tops.append(t)
#             if not top:
#                 valid_bottoms.append(t)

#         return valid_teams, valid_bottoms, valid_tops
    
    def generate_valid_teams(self):
        """Generate all valid pod combinations (aka, teams)"""

        valid_teams = []

        all_teams = itertools.combinations(self.pods, 3)
        for team in all_teams:
            bottom = max(pod.is_bottom for pod in team)
            top = max(pod.is_top for pod in team)
            if not(bottom and top):
                t = Team(pods=team)
                valid_teams.append(t)
        return valid_teams

    
#     def generate_weekly_teams(self):
#         """
#         We want to find the best schedule for the week subject to the
#         following constraints:
#         1. Potential team has never been used in a prior week
#         2. The deviation of score between the potential teams is as tight as can be
#         """
        
#         # First going to construct the teams that will play vs. tops (more restrictive)
#         pass
    
    def generate_matchups(self, collections):
        """
        Generate potential matchups by creating team1 vs. team2 combinations
        from valid teams.
        """

        matchups = dict()
        for ix, match in enumerate(itertools.product(*collections)):
            team1 = match[0]
            team2 = match[1]
            t1 = set(team1.pods)
            t2 = set(team2.pods)
            if len(t1.intersection(t2)) == 0:
                # Update the delta by the penalty of playing against another pod multiple times
                mean_played_against = team1.get_mean_played_against_team(team2)
                if mean_played_against > 0:
                    delta = int(round(
                        max(abs(team1.score - team2.score), 1) * (self.MU * mean_played_against),
                        0
                    ))
                else:
                    delta = abs(team1.score - team2.score)
                if delta in matchups:
                    matchups[delta].append((team1, team2))
                else:
                    matchups[delta] = [(team1, team2)]
        return matchups
    
    def generate_same_day_matchups(self, collections, schedules):
        """
        Generate potential matchups by creating team1 vs.team2 combinations
        from valid teams
        """
        
#         print(f'Input collections:')
#         print(collections)
#         print(f'Input schedules:')
#         print(schedules)
        matchups = dict()
        for ix, match in enumerate(itertools.product(*[collections, collections])):
            team1 = match[0]
            team2 = match[1]
#             print(f'matching: {team1} vs. {team2}')
            if not (
                (team1, team2) in [(event[0], event[1]) for event in schedules]
                or (team2, team1) in [(event[0], event[1]) for event in schedules]
            ):
                t1 = set(team1.pods)
                t2 = set(team2.pods)
                if len(t1.intersection(t2)) == 0:
                    # Update the delta by the penalty of playing against another pod multiple times
                    mean_played_against = team1.get_mean_played_against_team(team2)
                    if mean_played_against > 0:
#                         print('found a penalty!')
                        delta = int(round(
                            max(abs(team1.score - team2.score), 1) * (self.MU * mean_played_against), 
                            0
                        ))
#                         print(f'delta: {delta}')
                    else:
                        delta = abs(team1.score - team2.score)
#                         print(f'delta: {delta}')
                    if delta in matchups:
                        matchups[delta].append((team1, team2))
                    else:
                        matchups[delta] = [(team1, team2)]
            else:
#                 print(f'discarding: {team1} vs.{team2}')
                pass
        return matchups

    
    def initialize_assignments(self):
        """
        Initialize an empty assignment object.
        - Get unique pods from tops: this will be used each round to determine
          which pods yet must be assigned
        """
        assignments = dict()
        for pod in season.sorted_pods:
            if pod not in assignments:
                assignments[pod] = None
        return assignments
    
    def get_remaining_pods(self, assignments):
        return [pod for pod in assignments.keys() if assignments[pod] is None]
    
    def generate_first_schedule_of_day(self, matchups, assignments=None, debug=False):
        """
        Create a schedule of team1 vs. team2 for each team in a list. This is the 
        first matchup of a day so the teams can be composed of any pods.
        """
        # Get a list of deltas sorted from least to greatest
        deltas = list(matchups.keys())
        deltas.sort()
        
        # Initialize assignments
        if not assignments:
            assignments = self.initialize_assignments()
        
        # This is the score for the first match in a day
        # (score is the matching algorithm)
        # Initialize the function
        index = 0
        deltas = list(matchups.keys())
        deltas.sort()
        delta = deltas[0]
        cache = [(0, 0)]
        remaining = self.get_remaining_pods(assignments)
        bypass = False
        debug_cntr = 0
        schedule = []
        team_stack = []

        while len(remaining) > 0:

            # Debug print statements
            debug_cntr += 1
            if debug and debug_cntr % 100000000 == 0:
                print(f'Debug:{debug_cntr}\tIndex: {index}\tDelta: {delta}\tschedule:')
                for pair in schedule:
                    print(pair[0], pair[1])
                print(f'remaining: {len(remaining)}')

            # Get current match
            matches = matchups[delta]
            match = matches[index]

            # Validate whether this match can be added to the stack
            if not bypass:
                team1 = match[0]
                team2 = match[1]
                # Check for pod existence
                is_pod_exhausted = False
                for pod in team1.pods + team2.pods:
                    if pod not in remaining:
                        is_pod_exhausted = True
                        break
                if not is_pod_exhausted:
                    # schedule.append(match)
                    schedule.append((match[0], match[1], delta, (match[0].score, match[1].score)))
                    team_stack.append(team1)
                    team_stack.append(team2)
                    cache.append((index, delta))
                    # Add the pods to the assignments
                    for pod in team1.pods:
                        assignments[pod] = delta
                    for pod in team2.pods:
                        assignments[pod] = delta

            # Reset bypass to False in case it was set to True
            bypass = False

            # See if we solved it
            remaining = self.get_remaining_pods(assignments)

            # Schedule unfinished.
            # 1. Get the next index if an index exists
            index += 1
            if index >= len(matches):
                # We've exhaused the current delta. Get the next.
                try:
                    delta = min([d for d in list(matchups.keys()) if d > delta])
                    index = 0
                except ValueError:
                    # There is no next delta... reverse course
                    team1 = team_stack.pop()
                    team2 = team_stack.pop()
                    schedule.pop()
#                     print(f'popping cache: {cache}')
#                     print(f'cur index/delta: {index}::{delta}')
                    index, delta = cache.pop()
                    bypass = True
                    for pod in team1.pods:
                        assignments[pod] = None
                    for pod in team2.pods:
                        assignments[pod] = None
                    if index == 0 and delta == 0:
                        # We went back further than possible. Give up and die.
                        if debug:
                            print(f'Assignments:\n{assignments}\n\n')
                            print(f'schedule:\n{schedule}\n\n')
                            print(f'team_stack:\n{team_stack}\n\n')
                            print(f'cache:\n{cache}\n')
                        raise ValueError("We died.")

        # Outside the while loop means solved
        if debug:
            print('SOLVED')
        return schedule
        
    def generate_subsequent_schedule_of_day(
        self, schedules, debug=False
    ):
        """Create matches subsequent to the first matches of the day"""
        # Teams stay the same for a day so now we need to rotate the teams to play
        # against different teams but also make sure they minimize the count of times
        # they've played other pods previously.
                                        
        shuffle_teams = []
        for event in schedules:
            shuffle_teams.append(event[0])
            shuffle_teams.append(event[1])
        
        # matchups = self.generate_same_day_matchups(*[shuffle_teams, shuffle_teams], schedules)
        matchups = self.generate_same_day_matchups(shuffle_teams, schedules)
#         print(f'second matchups: {len(matchups)}')
#         print(list(matchups.keys()))
        
        # Get unique pods from tops: this will be used each round to determine which pods yet must be assigned
        assignments = self.initialize_assignments()
#         print('assignments:')
#         print(assignments)
        
        # Get the new schedule for the day
        schedule2 = self.generate_first_schedule_of_day(
            matchups=matchups, assignments=assignments, debug=debug
        )
        return schedule2
    
    def update_teams_played_against(self, team1, team2):
        # After the first event of the day, update the played_against schedule
        # for event in schedule:
        # team1 = event[0]
        # team2 = event[1]
        for p1 in team1.pods:
            for p2 in team2.pods:
                p1.play_against(p2)
                p2.play_against(p1)
    
    def update_schedule(self, week, day, game, schedule):
        if week in self.schedule:
            if day in self.schedule[week]:
                if game in self.schedule[week][day]:
                    for event in schedule:
                        self.schedule[week][day][game].append({
                            'team1': event[0],
                            'team2': event[1],
                            'delta': event[2],
                            'team1_rank': event[0].score,
                            'team2_rank': event[1].score
                        })
                        self.update_teams_played_against(event[0], event[1])
                else:
                    self.schedule[week][day][game] = []
                    for event in schedule:
                        self.schedule[week][day][game].append({
                            'team1': event[0],
                            'team2': event[1],
                            'delta': event[2],
                            'team1_rank': event[0].score,
                            'team2_rank': event[1].score
                        })
                        self.update_teams_played_against(event[0], event[1])
            else:
                self.schedule[week][day] = {game: []}
                for event in schedule:
                    self.schedule[week][day][game].append({
                        'team1': event[0],
                        'team2': event[1],
                        'delta': event[2],
                        'team1_rank': event[0].score,
                        'team2_rank': event[1].score
                    })
                    self.update_teams_played_against(event[0], event[1])
        else:
            self.schedule[week] = {day: {game: []}}
            for event in schedule:
                self.schedule[week][day][game].append({
                    'team1': event[0],
                    'team2': event[1],
                    'delta': event[2],
                    'team1_rank': event[0].score,
                    'team2_rank': event[1].score
                })
                self.update_teams_played_against(event[0], event[1])
        
        

In [98]:
# Generate teams and construct the world



In [99]:
# season = Season(pods=pods)

In [100]:
# len(season.pods)

In [101]:
pods = [Pod(name=ix, rank=ix) for ix in range(1, 49)]
season = Season(pods=pods)

# First round
matchups = season.generate_matchups([season.valid_teams, season.valid_teams])
schedule = season.generate_first_schedule_of_day(matchups=matchups, debug=True)
season.update_schedule(1, 1, 1, schedule)
for event in schedule:
    print(f'{event[0]} vs. {event[1]}')
    for pod in event[0].pods:
        print(f'pod: {pod} -> played against: {pod.played_against}')

KeyboardInterrupt: 

In [91]:
print(json.dumps(season.schedule[1][1][1], indent=2, default=str))

[
  {
    "team1": "[1, 2, 9]",
    "team2": "[3, 4, 5]",
    "delta": 0,
    "team1_rank": 12,
    "team2_rank": 12
  },
  {
    "team1": "[6, 7, 16]",
    "team2": "[8, 10, 11]",
    "delta": 0,
    "team1_rank": 29,
    "team2_rank": 29
  },
  {
    "team1": "[12, 13, 21]",
    "team2": "[14, 15, 17]",
    "delta": 0,
    "team1_rank": 46,
    "team2_rank": 46
  },
  {
    "team1": "[18, 19, 28]",
    "team2": "[20, 22, 23]",
    "delta": 0,
    "team1_rank": 65,
    "team2_rank": 65
  },
  {
    "team1": "[24, 25, 33]",
    "team2": "[26, 27, 29]",
    "delta": 0,
    "team1_rank": 82,
    "team2_rank": 82
  },
  {
    "team1": "[30, 31, 40]",
    "team2": "[32, 34, 35]",
    "delta": 0,
    "team1_rank": 101,
    "team2_rank": 101
  },
  {
    "team1": "[36, 38, 42]",
    "team2": "[37, 39, 41]",
    "delta": 1,
    "team1_rank": 116,
    "team2_rank": 117
  }
]


In [92]:
schedule2 = season.generate_subsequent_schedule_of_day(schedule, debug=True)

# season.update_schedule(1, 1, 2, schedule2)
# for event in schedule:
#     print(f'{event[0]} vs. {event[1]}')
#     for pod in event[0].pods:
#         print(f'pod: {pod} -> played against: {pod.played_against}')

Input collections:
[<__main__.Team object at 0x000001E96B502070>, <__main__.Team object at 0x000001E80395CEE0>, <__main__.Team object at 0x000001E81F842400>, <__main__.Team object at 0x000001E81D4CA040>, <__main__.Team object at 0x000001E703A5AC40>, <__main__.Team object at 0x000001E809ED18B0>, <__main__.Team object at 0x000001E83AB9D0D0>, <__main__.Team object at 0x000001E8425C1F10>, <__main__.Team object at 0x000001E842638B50>, <__main__.Team object at 0x000001E83AD0A730>, <__main__.Team object at 0x000001E846E02FD0>, <__main__.Team object at 0x000001E83AB23E50>, <__main__.Team object at 0x000001E830BF2F70>, <__main__.Team object at 0x000001E830BF59D0>]
Input schedules:
[(<__main__.Team object at 0x000001E96B502070>, <__main__.Team object at 0x000001E80395CEE0>, 0, (12, 12)), (<__main__.Team object at 0x000001E81F842400>, <__main__.Team object at 0x000001E81D4CA040>, 0, (29, 29)), (<__main__.Team object at 0x000001E703A5AC40>, <__main__.Team object at 0x000001E809ED18B0>, 0, (46, 46)

TypeError: 'int' object is not subscriptable

In [94]:
season.update_schedule(1, 1, 2, schedule2)

In [95]:
print(json.dumps(season.schedule[1][1][2], indent=2, default=str))

[
  {
    "team1": "[30, 31, 40]",
    "team2": "[36, 38, 42]",
    "delta": 15,
    "team1_rank": 101,
    "team2_rank": 116
  },
  {
    "team1": "[32, 34, 35]",
    "team2": "[37, 39, 41]",
    "delta": 16,
    "team1_rank": 101,
    "team2_rank": 117
  },
  {
    "team1": "[1, 2, 9]",
    "team2": "[6, 7, 16]",
    "delta": 17,
    "team1_rank": 12,
    "team2_rank": 29
  },
  {
    "team1": "[3, 4, 5]",
    "team2": "[8, 10, 11]",
    "delta": 17,
    "team1_rank": 12,
    "team2_rank": 29
  },
  {
    "team1": "[18, 19, 28]",
    "team2": "[24, 25, 33]",
    "delta": 17,
    "team1_rank": 65,
    "team2_rank": 82
  },
  {
    "team1": "[12, 13, 21]",
    "team2": "[20, 22, 23]",
    "delta": 19,
    "team1_rank": 46,
    "team2_rank": 65
  },
  {
    "team1": "[14, 15, 17]",
    "team2": "[26, 27, 29]",
    "delta": 36,
    "team1_rank": 46,
    "team2_rank": 82
  }
]


In [81]:
def test(schedules, debug=False):
    """Create matches subsequent to the first matches of the day"""
    # Teams stay the same for a day so now we need to rotate the teams to play
    # against different teams but also make sure they minimize the count of times
    # they've played other pods previously.

    shuffle_teams = []
    for event in schedules:
        print(f'{event[0]} vs. {event[1]}')
        shuffle_teams.append(event[0])
        shuffle_teams.append(event[1])

    # matchups = self.generate_same_day_matchups(*[shuffle_teams, shuffle_teams], schedules)
    matchups = season.generate_same_day_matchups(shuffle_teams, schedules)
    print(f'second matchups: {len(matchups)}')
    print(matchups)

    # Get unique pods from tops: this will be used each round to determine which pods yet must be assigned
    assignments = season.initialize_assignments()

    # Get the new schedule for the day
    schedule2 = season.generate_first_schedule_of_day(
        matchups=matchups, assignments=assignments, debug=debug
    )
    return schedule2
    

In [82]:
test(schedule, debug=True)

[1, 2, 9] vs. [3, 4, 5]
[6, 7, 16] vs. [8, 10, 11]
[12, 13, 21] vs. [14, 15, 17]
[18, 19, 28] vs. [20, 22, 23]
[24, 25, 33] vs. [26, 27, 29]
[30, 31, 40] vs. [32, 34, 35]
[36, 38, 42] vs. [37, 39, 41]
Input collections:
[<__main__.Team object at 0x000001E8426AA700>, <__main__.Team object at 0x000001E81FB50AC0>, <__main__.Team object at 0x000001E83D014F40>, <__main__.Team object at 0x000001E846D81B80>, <__main__.Team object at 0x000001E97E6D17F0>, <__main__.Team object at 0x000001E94E829610>, <__main__.Team object at 0x000001E83AC7FC70>, <__main__.Team object at 0x000001E81C289AF0>, <__main__.Team object at 0x000001E81C27D730>, <__main__.Team object at 0x000001E96B4F5310>, <__main__.Team object at 0x000001E96B4F4BB0>, <__main__.Team object at 0x000001E96B4DFA30>, <__main__.Team object at 0x000001E96B502B50>, <__main__.Team object at 0x000001E96B502FD0>]
Input schedules:
[(<__main__.Team object at 0x000001E8426AA700>, <__main__.Team object at 0x000001E81FB50AC0>, 0, (12, 12)), (<__main__

[(<__main__.Team at 0x1e96b4f4bb0>,
  15,
  (101, 116)),
 (<__main__.Team at 0x1e96b4dfa30>,
  16,
  (101, 117)),
 (<__main__.Team at 0x1e8426aa700>,
  17,
  (12, 29)),
 (<__main__.Team at 0x1e81fb50ac0>,
  17,
  (12, 29)),
 (<__main__.Team at 0x1e83ac7fc70>,
  17,
  (65, 82)),
 (<__main__.Team at 0x1e97e6d17f0>,
  19,
  (46, 65)),
 (<__main__.Team at 0x1e94e829610>,
  36,
  (46, 82))]

In [83]:
print(json.dumps(schedule2, indent=2, default=str))

[
  [
    "[30, 31, 40]",
    "[36, 38, 42]",
    15,
    [
      101,
      116
    ]
  ],
  [
    "[32, 34, 35]",
    "[37, 39, 41]",
    16,
    [
      101,
      117
    ]
  ],
  [
    "[1, 2, 9]",
    "[6, 7, 16]",
    17,
    [
      12,
      29
    ]
  ],
  [
    "[3, 4, 5]",
    "[8, 10, 11]",
    17,
    [
      12,
      29
    ]
  ],
  [
    "[18, 19, 28]",
    "[24, 25, 33]",
    17,
    [
      65,
      82
    ]
  ],
  [
    "[12, 13, 21]",
    "[20, 22, 23]",
    19,
    [
      46,
      65
    ]
  ],
  [
    "[14, 15, 17]",
    "[26, 27, 29]",
    36,
    [
      46,
      82
    ]
  ]
]


In [ ]:
***********************************************************

In [ ]:
GAMES_PER_DAY = 2
GAMES_PER_WEEK = 1
SCHEDULE_WEEKS = 6

pods = [Pod(name=ix, rank=ix) for ix in range(1, 49)]
season = Season(pods=pods)

for ix in range(0, SCHEDULE_WEEKS):
    for iy in range(0, GAMES_PER_WEEK):
        daily_schedules = []
        for iz in range(0, GAMES_PER_DAY):
            print(f'{ix}:{iy}:{iz}:::')
            if iz == 0:
                print('generating matchups...')
                matchups = season.generate_matchups([season.valid_teams, season.valid_teams])
                print('generating first schedule of day...')
                schedule = season.generate_first_schedule_of_day(matchups=matchups, debug=True)
#                 for event in schedule:
#                     print(f'{event[0]} vs. {event[1]}')
#                     for pod in event[0].pods:
#                         print(f'pod: {pod} -> played against: {pod.played_against}')
                season.update_schedule(ix + 1, iy + 1, iz + 1, schedule)
                daily_schedules += schedule
            else:
                print('generating second schedule of day...')
                schedule2 = season.generate_subsequent_schedule_of_day(schedule, debug=True)
#                 for event in schedule2:
#                     print(f'{event[0]} vs. {event[1]}')
                season.update_schedule(ix + 1, iy + 1, iz + 1, schedule2)
                daily_schedules += schedule2
            # Update the 'played with' for pods on the last game of the day
            if iz == len(GAMES_PER_DAY) - 1:
                for event in schedule2:
                    event[0].update_pods_played_with()
                    event[1].update_pods_played_with()
#             print('updating schedule...')
#             season.update_schedule(ix + 1, iy + 1, iz + 1, schedule)
#             print('***dailY-schedules:')
#             print(daily_schedules)
#             print('***first schedule')
#             print(schedule)
#             daily_schedules += schedule
#             print('***after dailY-sch:')
#             print(daily_schedules)
                

0:0:0:::
generating matchups...


In [108]:
import json

print(json.dumps(season.schedule[1][1][1], indent=2, default=str))

[
  {
    "team1": "[1, 2, 9]",
    "team2": "[3, 4, 5]",
    "delta": 0,
    "team1_rank": 12,
    "team2_rank": 12
  },
  {
    "team1": "[6, 7, 16]",
    "team2": "[8, 10, 11]",
    "delta": 0,
    "team1_rank": 29,
    "team2_rank": 29
  },
  {
    "team1": "[12, 13, 21]",
    "team2": "[14, 15, 17]",
    "delta": 0,
    "team1_rank": 46,
    "team2_rank": 46
  },
  {
    "team1": "[18, 19, 28]",
    "team2": "[20, 22, 23]",
    "delta": 0,
    "team1_rank": 65,
    "team2_rank": 65
  },
  {
    "team1": "[24, 25, 33]",
    "team2": "[26, 27, 29]",
    "delta": 0,
    "team1_rank": 82,
    "team2_rank": 82
  },
  {
    "team1": "[30, 31, 40]",
    "team2": "[32, 34, 35]",
    "delta": 0,
    "team1_rank": 101,
    "team2_rank": 101
  },
  {
    "team1": "[36, 38, 42]",
    "team2": "[37, 39, 41]",
    "delta": 1,
    "team1_rank": 116,
    "team2_rank": 117
  }
]


In [109]:
print(json.dumps(season.schedule[1][1][2], indent=2, default=str))

[
  {
    "team1": "[30, 31, 40]",
    "team2": "[36, 38, 42]",
    "delta": 15,
    "team1_rank": 101,
    "team2_rank": 116
  },
  {
    "team1": "[32, 34, 35]",
    "team2": "[37, 39, 41]",
    "delta": 16,
    "team1_rank": 101,
    "team2_rank": 117
  },
  {
    "team1": "[1, 2, 9]",
    "team2": "[6, 7, 16]",
    "delta": 17,
    "team1_rank": 12,
    "team2_rank": 29
  },
  {
    "team1": "[3, 4, 5]",
    "team2": "[8, 10, 11]",
    "delta": 17,
    "team1_rank": 12,
    "team2_rank": 29
  },
  {
    "team1": "[18, 19, 28]",
    "team2": "[24, 25, 33]",
    "delta": 17,
    "team1_rank": 65,
    "team2_rank": 82
  },
  {
    "team1": "[12, 13, 21]",
    "team2": "[20, 22, 23]",
    "delta": 19,
    "team1_rank": 46,
    "team2_rank": 65
  },
  {
    "team1": "[14, 15, 17]",
    "team2": "[26, 27, 29]",
    "delta": 36,
    "team1_rank": 46,
    "team2_rank": 82
  }
]


In [5]:
pod_to_match = pods[0]

for week in season.schedule.keys():
    for intraweek in season.schedule[week].keys():
        for intraday in season.schedule[week][intraweek].keys():
            matches = season.schedule[week][intraweek][intraday]
            for match in matches:
                if pod_to_match in match['team1'].pods:
                    print(
                        f"{match['team1']} vs. {match['team2']} ({match['delta']}):"
                        f"[{match['team1_rank']}-{match['team2_rank']}]"
                    )
                elif pod_to_match in match['team2'].pods:
                    print(
                        f"{match['team2']} vs. {match['team1']} ({match['delta']}):"
                        f"[{match['team2_rank']}-{match['team1_rank']}]"
                    )


[1, 2, 9] vs. [3, 4, 5] (0):[12-12]
[1, 2, 9] vs. [6, 7, 16] (17):[12-29]
[1, 2, 24] vs. [8, 9, 10] (0):[27-27]
[1, 2, 24] vs. [13, 14, 17] (17):[27-44]
[1, 2, 35] vs. [11, 12, 15] (0):[38-38]
[1, 2, 35] vs. [44, 45, 47] (98):[38-136]
[1, 19, 35] vs. [2, 21, 32] (0):[55-55]
[1, 19, 35] vs. [10, 26, 28] (225):[55-64]
[1, 31, 35] vs. [20, 23, 24] (0):[67-67]
[1, 31, 35] vs. [12, 22, 33] (42):[67-67]
[1, 6, 10] vs. [2, 5, 11] (67):[17-18]
[1, 6, 10] vs. [22, 40, 46] (758):[17-108]


In [117]:
pods[0].played_against

{<__main__.Pod at 0x1e840b36160>: {'count': 2},
 <__main__.Pod at 0x1e840b36a60>: {'count': 2},
 <__main__.Pod at 0x1e840b369a0>: {'count': 2},
 <__main__.Pod at 0x1e840b36a90>: {'count': 1},
 <__main__.Pod at 0x1e840b360a0>: {'count': 2},
 <__main__.Pod at 0x1e840b362e0>: {'count': 2},
 <__main__.Pod at 0x1e840b36130>: {'count': 2},
 <__main__.Pod at 0x1e840b36e50>: {'count': 1},
 <__main__.Pod at 0x1e840b36ac0>: {'count': 1},
 <__main__.Pod at 0x1e840b36670>: {'count': 1},
 <__main__.Pod at 0x1e840b36280>: {'count': 2},
 <__main__.Pod at 0x1e840b36700>: {'count': 1},
 <__main__.Pod at 0x1e840b36dc0>: {'count': 1},
 <__main__.Pod at 0x1e840b36d90>: {'count': 1},
 <__main__.Pod at 0x1e840b36430>: {'count': 1},
 <__main__.Pod at 0x1e840b36e20>: {'count': 1},
 <__main__.Pod at 0x1e840b36fa0>: {'count': 1},
 <__main__.Pod at 0x1e840b361f0>: {'count': 2},
 <__main__.Pod at 0x1e840b36460>: {'count': 1},
 <__main__.Pod at 0x1e840b36640>: {'count': 1},
 <__main__.Pod at 0x1e840b36c10>: {'coun

In [21]:
full_schedule = season.initialize_assignments()

for week in season.schedule.keys():
    for intraweek in season.schedule[week].keys():
        for intraday in season.schedule[week][intraweek].keys():
            matches = season.schedule[week][intraweek][intraday]
            for match in matches:
                if pod_to_match in match['team1'].pods:
                    print(
                        f"{match['team1']} vs. {match['team2']} ({match['delta']}):"
                        f"[{match['team1_rank']}-{match['team2_rank']}]"
                    )
                elif pod_to_match in match['team2'].pods:
                    print(
                        f"{match['team2']} vs. {match['team1']} ({match['delta']}):"
                        f"[{match['team2_rank']}-{match['team1_rank']}]"
                    )

[
  {
    "team1": "[1, 2, 18]",
    "team2": "[6, 7, 8]",
    "delta": 0,
    "team1_rank": 21,
    "team2_rank": 21
  },
  {
    "team1": "[3, 4, 19]",
    "team2": "[5, 10, 11]",
    "delta": 0,
    "team1_rank": 26,
    "team2_rank": 26
  },
  {
    "team1": "[9, 12, 29]",
    "team2": "[13, 16, 21]",
    "delta": 0,
    "team1_rank": 50,
    "team2_rank": 50
  },
  {
    "team1": "[14, 15, 30]",
    "team2": "[17, 20, 22]",
    "delta": 0,
    "team1_rank": 59,
    "team2_rank": 59
  },
  {
    "team1": "[23, 24, 41]",
    "team2": "[25, 31, 32]",
    "delta": 0,
    "team1_rank": 88,
    "team2_rank": 88
  },
  {
    "team1": "[26, 28, 42]",
    "team2": "[27, 34, 35]",
    "delta": 0,
    "team1_rank": 96,
    "team2_rank": 96
  },
  {
    "team1": "[33, 38, 40]",
    "team2": "[36, 37, 39]",
    "delta": 17,
    "team1_rank": 111,
    "team2_rank": 112
  }
]


In [132]:
full_schedule = season.initialize_assignments()
all_pods = [pod.name for pod in full_schedule.keys()]

columns = []
columns.append([None for x in range(0, 3)] + ['Pods'] + [None for x in range(0, len(all_pods))])
columns.append(['Week', 'Game of Week', 'Game of Day'] + [pod for pod in all_pods])

In [136]:
team_map = {}

team_counter = 1
for week in season.schedule.keys():
    for intraweek in season.schedule[week].keys():
        for intraday in season.schedule[week][intraweek].keys():
            matches = season.schedule[week][intraweek][intraday]
            pod_dict = {}
            for match in matches:
                # Allocate team1
                if match['team1'] in team_map:
                    # Team already exists so give it the same team number as previously granted
                    for pod in match['team1'].pods:
                        pod_dict[pod.name] = team_map[match['team1']]
                else:
                    for pod in match['team1'].pods:
                        pod_dict[pod.name] = team_counter
                    team_map[match['team1']] = team_counter
                    team_counter += 1
                # Allocate team2
                if match['team2'] in team_map:
                    # Team already exists so give it the same team number as previously granted
                    for pod in match['team2'].pods:
                        pod_dict[pod.name] = team_map[match['team2']]
                else:
                    for pod in match['team2'].pods:
                        pod_dict[pod.name] = team_counter
                    team_map[match['team2']] = team_counter
                    team_counter += 1
            # Add the lists to the columns object
            appending = [week, intraweek, intraday]
            for key in all_pods:
                appending.append(pod_dict[key])
            columns.append(appending)

In [130]:
pod_dict = {}
for pod in all_pods:
    pod_dict[pod] = None

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42]

In [ ]:
with open(r'G:\Users\bilquist\Documents\AndroidApps\josh\data\out.log', 'w') as f:
    for column in columns:
        f.writelines(str(column) + '\n')

In [ ]:
import pandas as pd

df = pd.DataFrame(columns)
# df.fillna('', inplace=True)

In [ ]:
dft = df.transpose()

In [ ]:
dft.to_csv(r'G:\Users\bilquist\Documents\AndroidApps\josh\data\out.csv', index=False)

In [146]:
df.transpose().head()

In [ ]:
# Pod table json

pod_map = {}
team_map = {}

SCHEDULE = []
POD = []
POD_PLAYED_WITH = []
POD_PLAYED_AGAINST = []
TEAM = []
TEAM_PODS = []
GAME = []

schedule_pk = 1
pod_pk = 1
team_pk = 1
team_pods_pk = 1
game_pk = 1
pod_played_with_pk = 1
pod_played_against_pk = 1



for week in season.schedule.keys():
    for intraweek in season.schedule[week].keys():
        for intraday in season.schedule[week][intraweek].keys():            
            matches = season.schedule[week][intraweek][intraday]
            for match in matches:
                map_team1_to_pods = False
                map_team2_to_pods = False
                
                # TEAMs
                # Assign the team1 a pk value
                if match['team1'] in team_map:
                    # Team already exists so use the same pk as previously granted
                    team1_id = team_map[match['team1']]
                else:
                    team1_id = team_pk
                    team_map[match['team1']] = team_pk
                    team_pk += 1
                    map_team1_to_pods = True
                TEAM.append({
                    "model": "core.Team",
                    "pk": team1_id,
                    "fields": {
                        "score": match['team1'].score,
                        "string_agg": str(match['team1'])
                    }
                })
                # Assign the team2 a pk value
                if match['team2'] in team_map:
                    # Team already exists so use the same pk as previously granted
                    team2_id = team_map[match['team2']]
                else:
                    team2_id = team_pk
                    team_map[match['team2']] = team_pk
                    team_pk += 1
                    map_team2_to_pods = True
                TEAM.append({
                    "model": "core.Team",
                    "pk": team2_id,
                    "fields": {
                        "score": match['team2'].score,
                        "string_agg": str(match['team2'])
                    }
                })
                
                # GAME
                # Assign the team1 and team2 to a game (team1 is home by default)
                game_id = game_pk
                GAME.append({
                    "model": "core.Game",
                    "pk": game_pk,
                    "fields": {
                        "home_team": team1_id,
                        "away_team": team2_id
                    }
                })
                game_pk += 1
                
                # SCHEDULE
                # Assign the game to a schedule
                SCHEDULE.append({
                    "model": "core.Schedule",
                    "pk": schedule_pk,
                    "fields": {
                        "week": week,
                        "game_of_week": intraweek,
                        "game_of_day": intraday,
                        "game": game_id
                    }
                })
                
                # PODs
                # Assign pods to model if not already exist (team1)
                team1_pod_ids = []
                for pod in match['team1'].pods:
                    if pod in pod_map:
                        pod_id = pod_map[pod]
                    else:
                        pod_id = pod_pk
                        pod_map[pod] = pod_pk
                        pod_pk += 1
                        POD.append({
                            "model": "core.Pod",
                            "pk": pod_id,
                            "fields": {
                                "name": pod.name,
                                "score": pod.rank
                            }
                        })
                    team1_pod_ids.append(pod_id)
                # Assign pods to model if not already exist (team2)
                team2_pod_ids = []
                for pod in match['team2'].pods:
                    if pod in pod_map:
                        pod_id = pod_map[pod]
                    else:
                        pod_id = pod_pk
                        pod_map[pod] = pod_pk
                        pod_pk += 1
                        POD.append({
                            "model": "core.Pod",
                            "pk": pod_id,
                            "fields": {
                                "name": pod.name,
                                "score": pod.rank
                            }
                        })
                    team2_pod_ids.append(pod_id)
                    
                # TEAMPODs
                if map_team1_to_pods:
                    for id_ in team1_pod_ids:
                        TEAM_PODS.append({
                            "model": "core.TeamPod",
                            "pk": team_pods_pk,
                            "fields": {
                                "team": team1_id,
                                "pod": id_
                            }
                        })
                        team_pods_pk += 1
                if map_team2_to_pods:
                    for id_ in team2_pod_ids:
                        TEAM_PODS.append({
                            "model": "core.TeamPod",
                            "pk": team_pods_pk,
                            "fields": {
                                "team": team2_id,
                                "pod": id_
                            }
                        })
                        team_pods_pk += 1
                    
                # PODPLAYEDAGAINST
                for pod1 in team1_pod_ids:
                    for pod2 in team2_pod_ids:
                        POD_PLAYED_AGAINST.append({
                            "model": "core.PodPlayedAgainst",
                            "pk": pod_played_against_pk,
                            "fields": {
                                "game": game_id,
                                "pod": pod1,
                                "opponent": pod2
                            }
                        })
                        pod_played_against_pk += 1
                        POD_PLAYED_AGAINST.append({
                            "model": "core.PodPlayedAgainst",
                            "pk": pod_played_against_pk,
                            "fields": {
                                "game": game_id,
                                "pod": pod2,
                                "opponent": pod1
                            }
                        })
                        pod_played_against_pk += 1
                
                # PODPLAYEDWITH
                # Team1
                for pod_couple in itertools.combinations(team1_pod_ids, 2):
                    pod1 = pod_couple[0]
                    pod2 = pod_couple[1]
                    POD_PLAYED_WITH.append({
                        "model": "core.PodPlayedWith",
                        "pk": pod_played_with_pk,
                        "fields": {
                            "game": game_id,
                            "pod": pod1,
                            "teammate": pod2
                        }
                    })
                    pod_played_with_pk += 1
                    POD_PLAYED_WITH.append({
                        "model": "core.PodPlayedWith",
                        "pk": pod_played_with_pk,
                        "fields": {
                            "game": game_id,
                            "pod": pod2,
                            "teammate": pod1
                        }
                    })
                    pod_played_with_pk += 1
                # Team2
                for pod_couple in itertools.combinations(team2_pod_ids, 2):
                    pod1 = pod_couple[0]
                    pod2 = pod_couple[1]
                    POD_PLAYED_WITH.append({
                        "model": "core.PodPlayedWith",
                        "pk": pod_played_with_pk,
                        "fields": {
                            "game": game_id,
                            "pod": pod1,
                            "teammate": pod2
                        }
                    })
                    pod_played_with_pk += 1
                    POD_PLAYED_WITH.append({
                        "model": "core.PodPlayedWith",
                        "pk": pod_played_with_pk,
                        "fields": {
                            "game": game_id,
                            "pod": pod2,
                            "teammate": pod1
                        }
                    })
                    pod_played_with_pk += 1


In [ ]:
vars(pods[-1])

In [11]:
# Get unique pods from tops: this will be used each round to determine which pods yet must be assigned
assignments = dict()

for pod in season.sorted_pods:
    if pod not in assignments:
        assignments[pod] = None


In [13]:
len(valid_teams)

9368

In [ ]:
with open(r'G:\Users\bilquist\Documents\AndroidApps\josh\data\schedule.json', 'w') as f:
    f.writelines(json.dumps(SCHEDULE, indent=2))
    
with open(r'G:\Users\bilquist\Documents\AndroidApps\josh\data\pod.json', 'w') as f:
    f.writelines(json.dumps(POD, indent=2))
    
with open(r'G:\Users\bilquist\Documents\AndroidApps\josh\data\pod_played_with.json', 'w') as f:
    f.writelines(json.dumps(POD_PLAYED_WITH, indent=2))
    
with open(r'G:\Users\bilquist\Documents\AndroidApps\josh\data\pod_played_against.json', 'w') as f:
    f.writelines(json.dumps(POD_PLAYED_AGAINST, indent=2))
    
with open(r'G:\Users\bilquist\Documents\AndroidApps\josh\data\team.json', 'w') as f:
    f.writelines(json.dumps(TEAM, indent=2))
    
with open(r'G:\Users\bilquist\Documents\AndroidApps\josh\data\team_pods.json', 'w') as f:
    f.writelines(json.dumps(TEAM_PODS, indent=2))
    
with open(r'G:\Users\bilquist\Documents\AndroidApps\josh\data\game.json', 'w') as f:
    f.writelines(json.dumps(GAME, indent=2))

In [66]:
import json

with open(r'G:\Users\bilquist\Documents\AndroidApps\josh\data\schedule.json', 'r') as f:
    SCHEDULE = json.loads(f.read())
    
with open(r'G:\Users\bilquist\Documents\AndroidApps\josh\data\pod.json', 'r') as f:
    POD = json.loads(f.read())
    
with open(r'G:\Users\bilquist\Documents\AndroidApps\josh\data\pod_played_with.json', 'r') as f:
    POD_PLAYED_WITH = json.loads(f.read())
    
with open(r'G:\Users\bilquist\Documents\AndroidApps\josh\data\pod_played_against.json', 'r') as f:
    POD_PLAYED_AGAINST = json.loads(f.read())
    
with open(r'G:\Users\bilquist\Documents\AndroidApps\josh\data\team.json', 'r') as f:
    TEAM = json.loads(f.read())

with open(r'G:\Users\bilquist\Documents\AndroidApps\josh\data\team_pods.json', 'r') as f:
    TEAM_PODS = json.loads(f.read())
    
with open(r'G:\Users\bilquist\Documents\AndroidApps\josh\data\game.json', 'r') as f:
    GAME = json.loads(f.read())

In [115]:
MOD_SCHEDULE = []
for event in SCHEDULE:
    new_event = {"id": event["pk"]}
    new_event.update(event["fields"])
    MOD_SCHEDULE.append(new_event)
    # MOD_SCHEDULE.append(event["fields"])
    
MOD_POD = []
for event in POD:
    new_event = {"id": event["pk"]}
    new_event.update(event["fields"])
    MOD_POD.append(new_event)
    # MOD_POD.append(event["fields"])

MOD_POD_PLAYED_WITH = []
for event in POD_PLAYED_WITH:
    new_event = {"id": event["pk"]}
    new_event.update(event["fields"])
    MOD_POD_PLAYED_WITH.append(new_event)
    # MOD_POD_PLAYED_WITH.append(event["fields"])

MOD_POD_PLAYED_AGAINST = []
for event in POD_PLAYED_AGAINST:
    new_event = {"id": event["pk"]}
    new_event.update(event["fields"])
    MOD_POD_PLAYED_AGAINST.append(new_event)
    # MOD_POD_PLAYED_AGAINST.append(event["fields"])

MOD_TEAM = []
for event in TEAM:
    new_event = {"id": event["pk"]}
    new_event.update(event["fields"])
    MOD_TEAM.append(new_event)
    # MOD_TEAM.append(event["fields"])

MOD_TEAM_PODS = []
for event in TEAM_PODS:
    new_event = {"id": event["pk"]}
    new_event.update(event["fields"])
    MOD_TEAM_PODS.append(new_event)
    # MOD_TEAM_PODS.append(event["fields"])

MOD_GAME = []
for event in GAME:
    new_event = {"id": event["pk"]}
    new_event.update(event["fields"])
    MOD_GAME.append(new_event)
    # MOD_GAME.append(event["fields"])

In [116]:
df_schedule = pd.read_json(json.dumps(MOD_SCHEDULE))

df_pod = pd.read_json(json.dumps(MOD_POD))

df_pod_played_with = pd.read_json(json.dumps(MOD_POD_PLAYED_WITH))

df_pod_played_against = pd.read_json(json.dumps(MOD_POD_PLAYED_AGAINST))

df_team = pd.read_json(json.dumps(MOD_TEAM))

df_team_pods = pd.read_json(json.dumps(MOD_TEAM_PODS))

df_game = pd.read_json(json.dumps(MOD_GAME))


In [117]:
df_schedule.head()

,id,week,game_of_week,game_of_day,game
0,1,1,1,1,1
1,1,1,1,1,2
2,1,1,1,1,3
3,1,1,1,1,4
4,1,1,1,1,5


In [118]:
df_pod.head()

,id,name,score
0,1,1,1
1,2,2,2
2,3,9,9
3,4,3,3
4,5,4,4


In [119]:
df_pod_played_with.head()

,id,game,pod,teammate
0,1,1,1,2
1,2,1,2,1
2,3,1,1,3
3,4,1,3,1
4,5,1,2,3


In [120]:
df_pod_played_against.head()

,id,game,pod,opponent
0,1,1,1,4
1,2,1,4,1
2,3,1,1,5
3,4,1,5,1
4,5,1,1,6


In [121]:
df_team.head()

,id,score,string_agg
0,1,12,"[1, 2, 9]"
1,2,12,"[3, 4, 5]"
2,3,29,"[6, 7, 16]"
3,4,29,"[8, 10, 11]"
4,5,46,"[12, 13, 21]"


In [122]:
df_team_pods

,id,team,pod
0,1,1,1
1,2,1,2
2,3,1,3
3,4,2,4
4,5,2,5
...,...,...,...
277,278,93,5
278,279,93,21
279,280,94,47
280,281,94,39


In [123]:
df_game.head()

,id,home_team,away_team
0,1,1,2
1,2,3,4
2,3,5,6
3,4,7,8
4,5,9,10


In [124]:
# Upload files to Cloud Storage

from google.cloud import storage
from google.oauth2 import service_account
import os

def upload_blob(bucket_name, source_file_name, destination_blob_name, credentials):
    """
    Uploads a file to the bucket:
    https://cloud.google.com/storage/docs/uploading-objects#storage-upload-object-python
    """
    
    storage_client = storage.Client(credentials=credentials)
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)
    
    blob.upload_from_filename(source_file_name)
    print(f'File {source_file_name} uploaded to {destination_blob_name}.')
    

In [125]:
# (List[Dict], Local file path, cloud storage file path)
to_upload = [
    (
        MOD_SCHEDULE,
        r'G:\Users\bilquist\Documents\AndroidApps\josh\data\gcp_schedule.json',
        'bigquery_staging/schedule.json'
    ),
    (
        MOD_POD,
        r'G:\Users\bilquist\Documents\AndroidApps\josh\data\gcp_pod.json',
        'bigquery_staging/pod.json'
    ),
    (
        MOD_POD_PLAYED_WITH,
        r'G:\Users\bilquist\Documents\AndroidApps\josh\data\gcp_pod_played_with.json',
        'bigquery_staging/pod_played_with.json'
    ),
    (
        MOD_POD_PLAYED_AGAINST,
        r'G:\Users\bilquist\Documents\AndroidApps\josh\data\gcp_pod_played_against.json',
        'bigquery_staging/pod_played_against.json'
    ),
    (
        MOD_TEAM,
        r'G:\Users\bilquist\Documents\AndroidApps\josh\data\gcp_team.json',
        'bigquery_staging/team.json'
    ),
    (
        MOD_TEAM_PODS,
        r'G:\Users\bilquist\Documents\AndroidApps\josh\data\gcp_team_pods.json',
        'bigquery_staging/team_pod.json'
    ),
    (
        MOD_GAME,
        r'G:\Users\bilquist\Documents\AndroidApps\josh\data\gcp_game.json',
        'bigquery_staging/game.json'
    )
]


# GCP credentials
key_path = os.path.join('..', 'secrets', 'bbpk2.json')
credentials = service_account.Credentials.from_service_account_file(key_path)

# Write json to disk and upload the files to GCS
BUCKET_NAME = 'frisbee-league'
for frame in to_upload:
    # Write the file to disk
    with open(frame[1], 'w') as f:
        for line in frame[0]:
            f.writelines(f'{json.dumps(line)}\n')
    # Upload the file to Cloud Storage
    upload_blob(BUCKET_NAME, frame[1], frame[2], credentials)

File G:\Users\bilquist\Documents\AndroidApps\josh\data\gcp_schedule.json uploaded to bigquery_staging/schedule.json.
File G:\Users\bilquist\Documents\AndroidApps\josh\data\gcp_pod.json uploaded to bigquery_staging/pod.json.
File G:\Users\bilquist\Documents\AndroidApps\josh\data\gcp_pod_played_with.json uploaded to bigquery_staging/pod_played_with.json.
File G:\Users\bilquist\Documents\AndroidApps\josh\data\gcp_pod_played_against.json uploaded to bigquery_staging/pod_played_against.json.
File G:\Users\bilquist\Documents\AndroidApps\josh\data\gcp_team.json uploaded to bigquery_staging/team.json.
File G:\Users\bilquist\Documents\AndroidApps\josh\data\gcp_team_pods.json uploaded to bigquery_staging/team_pod.json.
File G:\Users\bilquist\Documents\AndroidApps\josh\data\gcp_game.json uploaded to bigquery_staging/game.json.


In [94]:
# Load files to BigQuery

from google.cloud import bigquery


client = bigquery.Client(credentials=credentials)

In [127]:
# Schedule

job_config = bigquery.LoadJobConfig(
    schema=[
        bigquery.SchemaField("id", "INT64"),
        bigquery.SchemaField("week", "INT64"),
        bigquery.SchemaField("game_of_week", "INT64"),
        bigquery.SchemaField("game_of_day", "INT64"),
        bigquery.SchemaField("game", "INT64"),
    ],
    source_format=bigquery.SourceFormat.NEWLINE_DELIMITED_JSON,
    write_disposition=bigquery.WriteDisposition.WRITE_TRUNCATE
)
uri = 'gs://frisbee-league/bigquery_staging/schedule.json'
table_id = 'bluebird-safety.Ultimate.Schedule'

load_job = client.load_table_from_uri(
    uri, table_id, job_config=job_config
)

load_job.result()

destination_table = client.get_table(table_id)
print(f'Loaded {destination_table.num_rows} rows.')

Loaded 96 rows.


In [128]:
# Pod

job_config = bigquery.LoadJobConfig(
    schema=[
        bigquery.SchemaField("id", "INT64"),
        bigquery.SchemaField("name", "INT64"),  # TODO: Update to string
        bigquery.SchemaField("score", "INT64"),
    ],
    source_format=bigquery.SourceFormat.NEWLINE_DELIMITED_JSON,
    write_disposition=bigquery.WriteDisposition.WRITE_TRUNCATE
)
uri = 'gs://frisbee-league/bigquery_staging/pod.json'
table_id = 'bluebird-safety.Ultimate.Pod'

load_job = client.load_table_from_uri(
    uri, table_id, job_config=job_config
)

load_job.result()

destination_table = client.get_table(table_id)
print(f'Loaded {destination_table.num_rows} rows.')

Loaded 48 rows.


In [129]:
# Pod Played With

job_config = bigquery.LoadJobConfig(
    schema=[
        bigquery.SchemaField("id", "INT64"),
        bigquery.SchemaField("game", "INT64"),
        bigquery.SchemaField("pod", "INT64"),
        bigquery.SchemaField("teammate", "INT64"),
    ],
    source_format=bigquery.SourceFormat.NEWLINE_DELIMITED_JSON,
    write_disposition=bigquery.WriteDisposition.WRITE_TRUNCATE
)
uri = 'gs://frisbee-league/bigquery_staging/pod_played_with.json'
table_id = 'bluebird-safety.Ultimate.PodPlayedWith'

load_job = client.load_table_from_uri(
    uri, table_id, job_config=job_config
)

load_job.result()

destination_table = client.get_table(table_id)
print(f'Loaded {destination_table.num_rows} rows.')

Loaded 1152 rows.


In [130]:
# Pod Played Against

job_config = bigquery.LoadJobConfig(
    schema=[
        bigquery.SchemaField("id", "INT64"),
        bigquery.SchemaField("game", "INT64"),
        bigquery.SchemaField("pod", "INT64"),
        bigquery.SchemaField("opponent", "INT64"),
    ],
    source_format=bigquery.SourceFormat.NEWLINE_DELIMITED_JSON,
    write_disposition=bigquery.WriteDisposition.WRITE_TRUNCATE
)
uri = 'gs://frisbee-league/bigquery_staging/pod_played_against.json'
table_id = 'bluebird-safety.Ultimate.PodPlayedAgainst'

load_job = client.load_table_from_uri(
    uri, table_id, job_config=job_config
)

load_job.result()

destination_table = client.get_table(table_id)
print(f'Loaded {destination_table.num_rows} rows.')

Loaded 1728 rows.


In [131]:
# Team

job_config = bigquery.LoadJobConfig(
    schema=[
        bigquery.SchemaField("id", "INT64"),
        bigquery.SchemaField("score", "INT64"),
        bigquery.SchemaField("string_agg", "STRING"),
    ],
    source_format=bigquery.SourceFormat.NEWLINE_DELIMITED_JSON,
    write_disposition=bigquery.WriteDisposition.WRITE_TRUNCATE
)
uri = 'gs://frisbee-league/bigquery_staging/team.json'
table_id = 'bluebird-safety.Ultimate.Team'

load_job = client.load_table_from_uri(
    uri, table_id, job_config=job_config
)

load_job.result()

destination_table = client.get_table(table_id)
print(f'Loaded {destination_table.num_rows} rows.')

Loaded 192 rows.


In [135]:
# Team Pod

job_config = bigquery.LoadJobConfig(
    schema=[
        bigquery.SchemaField("id", "INT64"),
        bigquery.SchemaField("team", "INT64"),
        bigquery.SchemaField("pod", "INT64"),
    ],
    source_format=bigquery.SourceFormat.NEWLINE_DELIMITED_JSON,
    write_disposition=bigquery.WriteDisposition.WRITE_TRUNCATE
)
uri = 'gs://frisbee-league/bigquery_staging/team_pod.json'
table_id = 'bluebird-safety.Ultimate.TeamPod'

load_job = client.load_table_from_uri(
    uri, table_id, job_config=job_config
)

load_job.result()

destination_table = client.get_table(table_id)
print(f'Loaded {destination_table.num_rows} rows.')

Loaded 282 rows.


In [138]:
# Game

job_config = bigquery.LoadJobConfig(
    schema=[
        bigquery.SchemaField("id", "INT64"),
        bigquery.SchemaField("home_team", "INT64"),
        bigquery.SchemaField("away_team", "INT64"),
    ],
    source_format=bigquery.SourceFormat.NEWLINE_DELIMITED_JSON,
    write_disposition=bigquery.WriteDisposition.WRITE_TRUNCATE
)
uri = 'gs://frisbee-league/bigquery_staging/game.json'
table_id = 'bluebird-safety.Ultimate.Game'

load_job = client.load_table_from_uri(
    uri, table_id, job_config=job_config
)

load_job.result()

destination_table = client.get_table(table_id)
print(f'Loaded {destination_table.num_rows} rows.')

Loaded 96 rows.


In [137]:
MOD_GAME

[{'id': 1, 'home_team': 1, 'away_team': 2},
 {'id': 2, 'home_team': 3, 'away_team': 4},
 {'id': 3, 'home_team': 5, 'away_team': 6},
 {'id': 4, 'home_team': 7, 'away_team': 8},
 {'id': 5, 'home_team': 9, 'away_team': 10},
 {'id': 6, 'home_team': 11, 'away_team': 12},
 {'id': 7, 'home_team': 13, 'away_team': 14},
 {'id': 8, 'home_team': 15, 'away_team': 16},
 {'id': 9, 'home_team': 1, 'away_team': 3},
 {'id': 10, 'home_team': 2, 'away_team': 4},
 {'id': 11, 'home_team': 7, 'away_team': 9},
 {'id': 12, 'home_team': 8, 'away_team': 10},
 {'id': 13, 'home_team': 11, 'away_team': 13},
 {'id': 14, 'home_team': 12, 'away_team': 14},
 {'id': 15, 'home_team': 5, 'away_team': 15},
 {'id': 16, 'home_team': 6, 'away_team': 16},
 {'id': 17, 'home_team': 17, 'away_team': 18},
 {'id': 18, 'home_team': 19, 'away_team': 20},
 {'id': 19, 'home_team': 21, 'away_team': 22},
 {'id': 20, 'home_team': 23, 'away_team': 24},
 {'id': 21, 'home_team': 25, 'away_team': 26},
 {'id': 22, 'home_team': 27, 'away_team'

In [19]:
len(list(itertools.combinations(pods, 3)))

11480

In [20]:
len(season.generate_potential_teams())

3

In [21]:
[x.score for x in season.valid_teams[:5]] + [x.score for x in season.valid_teams[-5:]]

[6, 7, 8, 9, 10, 121, 120, 121, 122, 123]

In [22]:
season.valid_teams[0].score

6

In [23]:
season.valid_teams[1000].score

61

In [24]:
len(season.valid_teams)

9368

In [25]:
set(season.valid_teams[0].pods)

{<__main__.Pod at 0x236f6085df0>,
 <__main__.Pod at 0x236f6085a90>}

In [71]:
teams1 = []
teams2 = []

for ix, match in enumerate(itertools.product(*[season.valid_teams, season.valid_teams])):
    t1 = set(match[0].pods)
    t2 = set(match[1].pods)
    if len(t1.intersection(t2)) == 0:
        teams1.append(match[0])
        teams2.append(match[1])
        

In [21]:
import pandas as pd

df = pd.DataFrame(
    {
        'team1': teams1, 
        'team2': teams2,
        'score1': [team.score for team in teams1],
        'score2': [team.score for team in teams2],
        'pod11': [tuple(team.pods)[0] for team in teams1],
        'pod12': [tuple(team.pods)[1] for team in teams1],
        'pod13': [tuple(team.pods)[2] for team in teams1],
        'pod21': [tuple(team.pods)[0] for team in teams2],
        'pod22': [tuple(team.pods)[1] for team in teams2],
        'pod23': [tuple(team.pods)[2] for team in teams2],
    }
)

In [22]:
df['delta'] = abs(df.score1 - df.score2)

In [23]:
df.sort_values(['delta'], inplace=True)

In [24]:
df.head()

,team1,team2,score1,score2,pod11,pod12,pod13,pod21,pod22,pod23,delta
8995679,"[42, 35, 12]","[25, 27, 37]",89,89,42,35,12,25,27,37,0
8539008,"[41, 12, 26]","[28, 35, 16]",79,79,41,12,26,28,35,16,0
11773100,"[22, 38, 14]","[25, 39, 10]",74,74,22,38,14,25,39,10,0
2303536,"[9, 38, 28]","[25, 14, 36]",75,75,9,38,28,25,14,36,0
15402570,"[35, 30, 16]","[38, 24, 19]",81,81,35,30,16,38,24,19,0


In [102]:
# Now to iterate through the season to assign actual matches based on the following constraints:
# 1. Never be in the same pod as another team more than once
# 2. Only play against opposing pods, at maximum, three times

total_games = season.GAMES_PER_WEEK * season.SCHEDULE_WEEKS

# Each pod object manages its own cache to track whom they've played with and against
# but we need to know which pods have been assigned this week
matches = dict(zip(pods, [0] * len(pods)))

# Start with valid_tops
for team in season.valid_tops:
    # Find the team in the df in position team1 or position team2 which has the lowest delta,
    # such that the opposing team has not yet been assigned, and such that the teams are constructed
    # from pods that have not yet been assigned with one-another
    
    # 1. Determine if these pods have ever played with one-another
    if team.pods[1] in team.pods[0].played_with or team.pods[2] in team.pods[0].played_with:
        # Pods have already played with one-another
        pass
    
    
    for pod in team.pods:
        if pod.played_with


In [28]:
for pod in season.valid_tops[0].pods:
    print(pod)

9
11
10


In [155]:
my_pod = pods[0]

print(my_pod.name)
for ix, row in df.iterrows():
    print(row['team1'])
    print(row['team2'])
    if my_pod in row['team1'].pods or my_pod in row['team2'].pods:
        print(True)
    else:
        print(False)
    break

1
[35, 12, 42]
[27, 37, 25]
False


In [133]:
df['team1'][0].pods.contains(season.valid_tops[0])

AttributeError: 'set' object has no attribute 'contains'

In [150]:
df[['team1', 'team2']].apply(lambda x: pods[0] in x, axis=1)

KeyboardInterrupt: 

In [139]:
pods[0] in season.valid_tops[0].pods

False

<h1> start new </h1>

In [30]:
# Create first matchup for a given day

matchups = dict()
mu = 50


for ix, match in enumerate(itertools.product(*[season.valid_teams, season.valid_teams])):
    team1 = match[0]
    team2 = match[1]
    t1 = set(team1.pods)
    t2 = set(team2.pods)
    if len(t1.intersection(t2)) == 0:
#         delta = abs(team1.score - team2.score)
#         if delta in matchups:
#             matchups[delta].append((team1, team2))
#         else:
#             matchups[delta] = [(team1, team2)]
        # Update the delta by the penalty of playing against another pod multiple times
        mean_played_against = team1.get_mean_played_against_team(team2)
        if mean_played_against > 0:
            delta = int(round(max(abs(team1.score - team2.score), 1) * (mu * mean_played_against), 0))
        else:
            delta = abs(team1.score - team2.score)
        if delta in matchups:
            matchups[delta].append((team1, team2))
        else:
            matchups[delta] = [(team1, team2)]


In [31]:
# Get a list of deltas sorted from least to greatest

deltas = list(matchups.keys())
deltas.sort()

In [32]:
# Get unique pods from tops: this will be used each round to determine which pods yet must be assigned

assignments = dict()

for pod in season.sorted_pods:
    if pod not in assignments:
        assignments[pod] = None


In [33]:
def get_remaining_pods(assignments):
    return [pod for pod in assignments.keys() if assignments[pod] is None]

In [34]:
# This is the score for the first match in a day
# (score is the matching algorithm)

def score(matchups, assignments):
    
    # Initialize the function
    index = 0
    deltas = list(matchups.keys())
    deltas.sort()
    delta = deltas[0]
    cache = [(0, 0)]
    remaining = get_remaining_pods(assignments)
    bypass = False
    debug = 0
    schedule = []
    team_stack = []
    
    while len(remaining) > 0:
        
        # Debug print statements
        debug += 1
        if debug % 100000 == 0:
#         if True:
            print(f'Debug:{debug}\tIndex: {index}\tDelta: {delta}\tschedule:')
            for pair in schedule:
                print(pair[0], pair[1])
            print(f'remaining: {len(remaining)}')
#         elif debug >= 10:
#             return schedule
#         print(f'Debug:{debug}\tIndex: {index}\tDelta: {delta}\tschedule:')
#         for pair in schedule:
#             print(pair[0], pair[1])
#         print(f'remaining: {len(remaining)}')
        
        # Get current match
        matches = matchups[delta]
        match = matches[index]
        
        # Validate whether this match can be added to the stack
        if not bypass:
            team1 = match[0]
            team2 = match[1]
            # Check for pod existence
            is_pod_exhausted = False
            for pod in team1.pods + team2.pods:
                if pod not in remaining:
                    is_pod_exhausted = True
                    break
            if not is_pod_exhausted:
                # schedule.append(match)
                schedule.append((match[0], match[1], delta, (match[0].score, match[1].score)))
                team_stack.append(team1)
                team_stack.append(team2)
                cache.append((index, delta))
                # Add the pods to the assignments
                for pod in team1.pods:
                    assignments[pod] = delta
                for pod in team2.pods:
                    assignments[pod] = delta
        
        # Reset bypass to False in case it was set to True
        bypass = False
        
        # See if we solved it
        remaining = get_remaining_pods(assignments)
        
        # Schedule unfinished.
        # 1. Get the next index if an index exists
        index += 1
        if index >= len(matches):
            # We've exhaused the current delta. Get the next.
            try:
                delta = min([d for d in list(matchups.keys()) if d > delta])
                index = 0
            except ValueError:
                # There is no next delta... reverse course
                team1 = team_stack.pop()
                team2 = team_stack.pop()
                schedule.pop()
                print(f'popping cache: {cache}')
                print(f'cur index/delta: {index}::{delta}')
                index, delta = cache.pop()
                bypass = True
                for pod in team1.pods:
                    assignments[pod] = None
                for pod in team2.pods:
                    assignments[pod] = None
                if index == 0 and delta == 0:
                    # We went back further than possible. Give up and die.
                    print(f'Assignments:\n{assignments}\n\n')
                    print(f'schedule:\n{schedule}\n\n')
                    print(f'team_stack:\n{team_stack}\n\n')
                    print(f'cache:\n{cache}\n')
                    raise ValueError("We died.")
    
    # Outside the while loop means solved
    print('SOLVED')
    return schedule


In [35]:
schedule = score(
    matchups=matchups,
    assignments=assignments
)

Debug:100000	Index: 99999	Delta: 150	schedule:
[1, 2, 8] [3, 4, 5]
remaining: 36
Debug:200000	Index: 199999	Delta: 150	schedule:
[1, 2, 8] [3, 4, 5]
remaining: 36
Debug:300000	Index: 299999	Delta: 150	schedule:
[1, 2, 8] [3, 4, 5]
remaining: 36
Debug:400000	Index: 399999	Delta: 150	schedule:
[1, 2, 8] [3, 4, 5]
remaining: 36
Debug:500000	Index: 499999	Delta: 150	schedule:
[1, 2, 8] [3, 4, 5]
remaining: 36
Debug:600000	Index: 599999	Delta: 150	schedule:
[1, 2, 8] [3, 4, 5]
[6, 7, 16] [9, 10, 11]
remaining: 30
Debug:700000	Index: 699999	Delta: 150	schedule:
[1, 2, 8] [3, 4, 5]
[6, 7, 16] [9, 10, 11]
remaining: 30
Debug:800000	Index: 799999	Delta: 150	schedule:
[1, 2, 8] [3, 4, 5]
[6, 7, 16] [9, 10, 11]
remaining: 30
Debug:900000	Index: 899999	Delta: 150	schedule:
[1, 2, 8] [3, 4, 5]
[6, 7, 16] [9, 10, 11]
remaining: 30
Debug:1000000	Index: 999999	Delta: 150	schedule:
[1, 2, 8] [3, 4, 5]
[6, 7, 16] [9, 10, 11]
remaining: 30
Debug:1100000	Index: 1099999	Delta: 150	schedule:
[1, 2, 8] [3, 4

In [36]:
for event in schedule:
    print(f'{event[0]} vs. {event[1]} ::: {event[2]} {event[3]}')

[1, 2, 8] vs. [3, 4, 5] ::: 150 (11, 12)
[6, 7, 16] vs. [9, 10, 11] ::: 150 (29, 30)
[12, 13, 20] vs. [14, 15, 17] ::: 150 (45, 46)
[18, 19, 28] vs. [21, 22, 23] ::: 150 (65, 66)
[24, 25, 32] vs. [26, 27, 29] ::: 150 (81, 82)
[30, 31, 40] vs. [33, 34, 35] ::: 150 (101, 102)
[36, 38, 42] vs. [37, 39, 41] ::: 150 (116, 117)


<h1>Second round on day</h1>

In [37]:
# Teams stay the same for a day so now we need to rotate the teams to play
# against different teams but also make sure they minimize the count of times
# they've played other pods previously.

# After the first event of the day, update the played_against schedule
for event in schedule:
    team1 = event[0]
    team2 = event[1]
    for p1 in team1.pods:
        for p2 in team2.pods:
            p1.play_against(p2)
            p2.play_against(p1)

In [64]:
shuffle_teams = []
for event in schedule:
    shuffle_teams.append(event[0])
    shuffle_teams.append(event[1])

In [65]:
matchups = dict()
mu = 50

for ix, match in enumerate(itertools.product(*[shuffle_teams, shuffle_teams])):
    team1 = match[0]
    team2 = match[1]
    if not (
        (team1, team2) in [(event[0], event[1]) for event in schedule]
        or (team2, team1) in [(event[0], event[1]) for event in schedule]
    ):
        t1 = set(team1.pods)
        t2 = set(team2.pods)
        if len(t1.intersection(t2)) == 0:
            # Update the delta by the penalty of playing against another pod multiple times
            mean_played_against = team1.get_mean_played_against_team(team2)
            if mean_played_against > 0:
                delta = int(round(max(abs(team1.score - team2.score), 1) * (mu * mean_played_against), 0))
            else:
                delta = abs(team1.score - team2.score)
            if delta in matchups:
                matchups[delta].append((team1, team2))
            else:
                matchups[delta] = [(team1, team2)]


In [69]:
len(matchups)

19

In [67]:
# Get unique pods from tops: this will be used each round to determine which pods yet must be assigned
assignments = dict()

for pod in season.sorted_pods:
    if pod not in assignments:
        assignments[pod] = None


schedule2 = season.generate_first_schedule_of_day(
    matchups=matchups,
    assignments=assignments
)

popping cache: [(0, 0), (0, 15), (1, 16), (0, 17), (3, 17), (16, 17), (19, 17)]
cur index/delta: 4::105
popping cache: [(0, 0), (0, 15), (1, 16), (0, 17), (3, 17), (16, 17), (23, 17)]
cur index/delta: 4::105


In [68]:
for ix, event in enumerate(schedule2):
    if max([p.name == 1 for p in event[0].pods]):
        print(f'{event[0]} {event[1]}')
    if max([p.name == 1 for p in event[1].pods]):
        print(f'{event[1]} {event[0]}')


[1, 2, 9] [6, 7, 16]


In [42]:
print(schedule[0][0], schedule[0][1])

[1, 2, 8] [3, 4, 5]


In [44]:
for event in schedule2:
    print(f'{event[0]} vs. {event[1]} ::: {event[2]} {event[3]}')

[33, 34, 35] vs. [36, 38, 42] ::: 2100 (102, 116)
[9, 10, 11] vs. [12, 13, 20] ::: 2250 (30, 45)
[21, 22, 23] vs. [24, 25, 32] ::: 2250 (66, 81)
[30, 31, 40] vs. [37, 39, 41] ::: 2400 (101, 117)
[3, 4, 5] vs. [6, 7, 16] ::: 2550 (12, 29)
[18, 19, 28] vs. [26, 27, 29] ::: 2550 (65, 82)
[1, 2, 8] vs. [14, 15, 17] ::: 5250 (11, 46)


<h1>Subsequent Matches</h1>

In [70]:
# After the first event of the day, update the played_against schedule

for event in schedule:
    team1 = event[0]
    team2 = event[1]
    for p1 in team1.pods:
        for p2 in team2.pods:
            p1.play_against(p2)
            p2.play_against(p1)

In [73]:
test = schedule[0][1].pods
test[0].played_against

{<__main__.Pod at 0x17bfcd55220>: {'count': 1},
 <__main__.Pod at 0x17bfcd55910>: {'count': 1},
 <__main__.Pod at 0x17bfcd55cd0>: {'count': 1}}

In [108]:
matchups = dict()
mu = 50

for ix, match in enumerate(itertools.product(*[season.valid_teams, season.valid_teams])):
    team1 = match[0]
    team2 = match[1]
    t1 = set(team1.pods)
    t2 = set(team2.pods)
    if len(t1.intersection(t2)) == 0:
        # Update the delta by the penalty of playing against another pod multiple times
        mean_played_against = team1.get_mean_played_against_team(team2)
        if mean_played_against > 0:
            delta = int(round(max(abs(team1.score - team2.score), 1) * (mu * mean_played_against), 0))
        else:
            delta = abs(team1.score - team2.score)
        if (
            (team1 == schedule[0][0] and team2 == schedule[0][1])
            or (team1 == schedule[0][1] and team2 == schedule[0][0])
        ): 
            print(f'delta: {delta}')
            print(f'mu: {mu}')
            print(f'mean_played_against: {mean_played_against}')
        if delta in matchups:
            matchups[delta].append((team1, team2))
        else:
            matchups[delta] = [(team1, team2)]

delta: 75
mu: 50
mean_played_against: 1.5
delta: 75
mu: 50
mean_played_against: 1.5


In [110]:
# Get unique pods from tops: this will be used each round to determine which pods yet must be assigned
assignments = dict()

for pod in season.sorted_pods:
    if pod not in assignments:
        assignments[pod] = None


schedule2 = score(
    matchups=matchups,
    assignments=assignments
)

Debug:100000	Index: 99999	Delta: 0	schedule:
[1, 2, 18] [6, 7, 8]
[3, 4, 19] [5, 10, 11]
[9, 12, 29] [13, 16, 21]
remaining: 24
Debug:200000	Index: 199999	Delta: 0	schedule:
[1, 2, 18] [6, 7, 8]
[3, 4, 19] [5, 10, 11]
[9, 12, 29] [13, 16, 21]
[14, 15, 30] [17, 20, 22]
remaining: 18
Debug:300000	Index: 10291	Delta: 1	schedule:
[1, 2, 18] [6, 7, 8]
[3, 4, 19] [5, 10, 11]
[9, 12, 29] [13, 16, 21]
[14, 15, 30] [17, 20, 22]
[23, 24, 41] [25, 31, 32]
[26, 28, 42] [27, 34, 35]
remaining: 6
Debug:400000	Index: 110291	Delta: 1	schedule:
[1, 2, 18] [6, 7, 8]
[3, 4, 19] [5, 10, 11]
[9, 12, 29] [13, 16, 21]
[14, 15, 30] [17, 20, 22]
[23, 24, 41] [25, 31, 32]
[26, 28, 42] [27, 34, 35]
remaining: 6
Debug:500000	Index: 210291	Delta: 1	schedule:
[1, 2, 18] [6, 7, 8]
[3, 4, 19] [5, 10, 11]
[9, 12, 29] [13, 16, 21]
[14, 15, 30] [17, 20, 22]
[23, 24, 41] [25, 31, 32]
[26, 28, 42] [27, 34, 35]
remaining: 6
Debug:600000	Index: 310291	Delta: 1	schedule:
[1, 2, 18] [6, 7, 8]
[3, 4, 19] [5, 10, 11]
[9, 12, 29

Debug:4200000	Index: 407149	Delta: 7	schedule:
[1, 2, 18] [6, 7, 8]
[3, 4, 19] [5, 10, 11]
[9, 12, 29] [13, 16, 21]
[14, 15, 30] [17, 20, 22]
[23, 24, 41] [25, 31, 32]
[26, 28, 42] [27, 34, 35]
remaining: 6
Debug:4300000	Index: 507149	Delta: 7	schedule:
[1, 2, 18] [6, 7, 8]
[3, 4, 19] [5, 10, 11]
[9, 12, 29] [13, 16, 21]
[14, 15, 30] [17, 20, 22]
[23, 24, 41] [25, 31, 32]
[26, 28, 42] [27, 34, 35]
remaining: 6
Debug:4400000	Index: 20849	Delta: 8	schedule:
[1, 2, 18] [6, 7, 8]
[3, 4, 19] [5, 10, 11]
[9, 12, 29] [13, 16, 21]
[14, 15, 30] [17, 20, 22]
[23, 24, 41] [25, 31, 32]
[26, 28, 42] [27, 34, 35]
remaining: 6
Debug:4500000	Index: 120849	Delta: 8	schedule:
[1, 2, 18] [6, 7, 8]
[3, 4, 19] [5, 10, 11]
[9, 12, 29] [13, 16, 21]
[14, 15, 30] [17, 20, 22]
[23, 24, 41] [25, 31, 32]
[26, 28, 42] [27, 34, 35]
remaining: 6
Debug:4600000	Index: 220849	Delta: 8	schedule:
[1, 2, 18] [6, 7, 8]
[3, 4, 19] [5, 10, 11]
[9, 12, 29] [13, 16, 21]
[14, 15, 30] [17, 20, 22]
[23, 24, 41] [25, 31, 32]
[26, 

Debug:8200000	Index: 500729	Delta: 12	schedule:
[1, 2, 18] [6, 7, 8]
[3, 4, 19] [5, 10, 11]
[9, 12, 29] [13, 16, 21]
[14, 15, 30] [17, 20, 22]
[23, 24, 41] [25, 31, 32]
[26, 28, 42] [27, 34, 35]
remaining: 6
Debug:8300000	Index: 16927	Delta: 13	schedule:
[1, 2, 18] [6, 7, 8]
[3, 4, 19] [5, 10, 11]
[9, 12, 29] [13, 16, 21]
[14, 15, 30] [17, 20, 22]
[23, 24, 41] [25, 31, 32]
[26, 28, 42] [27, 34, 35]
remaining: 6
Debug:8400000	Index: 116927	Delta: 13	schedule:
[1, 2, 18] [6, 7, 8]
[3, 4, 19] [5, 10, 11]
[9, 12, 29] [13, 16, 21]
[14, 15, 30] [17, 20, 22]
[23, 24, 41] [25, 31, 32]
[26, 28, 42] [27, 34, 35]
remaining: 6
Debug:8500000	Index: 216927	Delta: 13	schedule:
[1, 2, 18] [6, 7, 8]
[3, 4, 19] [5, 10, 11]
[9, 12, 29] [13, 16, 21]
[14, 15, 30] [17, 20, 22]
[23, 24, 41] [25, 31, 32]
[26, 28, 42] [27, 34, 35]
remaining: 6
Debug:8600000	Index: 316927	Delta: 13	schedule:
[1, 2, 18] [6, 7, 8]
[3, 4, 19] [5, 10, 11]
[9, 12, 29] [13, 16, 21]
[14, 15, 30] [17, 20, 22]
[23, 24, 41] [25, 31, 32]


Debug:12200000	Index: 1594425	Delta: 17	schedule:
[1, 2, 18] [6, 7, 8]
[3, 4, 19] [5, 10, 11]
[9, 12, 29] [13, 16, 21]
[14, 15, 30] [17, 20, 22]
[23, 24, 41] [25, 31, 32]
[26, 28, 42] [27, 34, 35]
remaining: 6
SOLVED


In [123]:
for ix, event in enumerate(schedule2):
    if max([p.name == 1 for p in event[0].pods]):
        print(f'1: {event[0]}')
    if max([p.name == 1 for p in event[1].pods]):
        print(f'1: {event[1]}')


1: [1, 2, 18]


In [119]:
print(schedule2[0][0])

[1, 2, 18]


In [124]:
print(schedule[0][0])

[1, 2, 9]


In [100]:
schedule[0][1]

In [102]:
rt = 0
for p1 in schedule[0][0].pods:
    for p2 in schedule[0][1].pods:
        if p2 in p1.played_against:
            rt += p1.played_against[p2].get('count', 0)
print(rt)

9


In [99]:
def score(
    matchups, assignments, index=None, delta=None, schedule=[], team_stack=[], cache=[], 
    bypass=False, debug=0, remaining=None
):
    # Initialization case
    if index is None:
        index = 0
        delta = list(matchups.keys())[0]
        cache.append((index, delta))
        remaining = get_remaining_pods(assignments)

    # Debug print statements
    debug += 1
    if debug % 10000 == 0:
        print(f'Debug:{debug}\tIndex: {index}\tDelta: {delta}\tschedule:')
        for pair in schedule:
            print(pair[0], pair[1])
        print(f'remaining:')
        print(remaining)
    elif debug >= 10:
        return schedule
    print(f'Debug:{debug}\tIndex: {index}\tDelta: {delta}\tschedule:')
    for pair in schedule:
        print(pair[0], pair[1])
    print(f'remaining:')
    print(remaining)
    
    # Get current match
    matches = matchups[delta]
    match = matches[index]
    
    # Validate whether this match can be added to the stack
    if not bypass:
        team1 = match[0]
        print(team1)
        team2 = match[1]
        print(team2)
        # Check same team existence
        if team1 not in team_stack and team2 not in team_stack:
            # Check for pod existence
            is_pod_exhausted = False
            for pod in team1.pods + team2.pods:
                print(f'Checking pod: {pod}')
                if pod not in remaining:
                    is_pod_exhausted = True
                    break
            if not is_pod_exhausted:
                print(f'Pods not exhaused. Adding to schedule.')
                schedule.append(match)
                team_stack.append(team1)
                team_stack.append(team2)
                # Add the pods to the assignments
                for pod in team1.pods:
                    assignments[pod] = delta
                for pod in team2.pods:
                    assignments[pod] = delta
    # Reset bypass to False in case it was set to True
    bypass = False
    
    # See if we solved it
    remaining = get_remaining_pods(assignments)
    # Base case
    if len(remaining) == 0:
        print('SOLVED!')
        return schedule
    
    # Schedule unfinished.
    # 1. Get the next index if an index exists
    index += 1
    if index >= len(matches):
        # We've exhaused the current delta. Get the next.
        try:
            next_delta = min([d for d in list(matchups.keys()) if d > delta])
            index = 0
            score(
                matchups, assignments, index, delta, schedule, team_stack, cache, bypass, debug,
                remaining
            )
        except ValueError:
            # There is no next delta... reverse course
            team1 = team_stack.pop()
            team2 = team_stack.pop()
            match = schedule.pop()
            new_index, new_delta = cache.pop()
            for pod in team1.pods:
                assignments[pod] = None
            for pod in team2.pods:
                assignments[pod] = None
            if new_index == 0 and new_delta == 0:
                # We went back further than possible. Give up and die.
                print(f'Assignments:\n{assignments}\n\n')
                print(f'schedule:\n{schedule}\n\n')
                print(f'team_stack:\n{team_stack}\n\n')
                print(f'cache:\n{cache}\n')
                raise ValueError("We died.")
            score(
                matchups, assignments, index, delta, schedule, team_stack, cache, bypass, debug, 
                remaining
            )
    else:
        score(
            matchups, assignments, index, delta, schedule, team_stack, cache, bypass, debug, 
            remaining
        )


In [17]:
schedule = score(
    matchups=matchups,
    assignments=assignments
)

Debug:1	Index: 0	Delta: 9	schedule:
remaining:
[<__main__.Pod object at 0x0000017BFCD55220>, <__main__.Pod object at 0x0000017BFCD55910>, <__main__.Pod object at 0x0000017BFCD55AF0>, <__main__.Pod object at 0x0000017BFCD55730>, <__main__.Pod object at 0x0000017BFCD55E80>, <__main__.Pod object at 0x0000017BFCD55F40>, <__main__.Pod object at 0x0000017BFCD55FD0>, <__main__.Pod object at 0x0000017BFCD55B80>, <__main__.Pod object at 0x0000017BFCD55CD0>, <__main__.Pod object at 0x0000017BFCD55D00>, <__main__.Pod object at 0x0000017BFCD55280>, <__main__.Pod object at 0x0000017BFCD55C70>, <__main__.Pod object at 0x0000017BFCD55760>, <__main__.Pod object at 0x0000017BFCD55820>, <__main__.Pod object at 0x0000017BFCD55940>, <__main__.Pod object at 0x0000017BFCD55580>, <__main__.Pod object at 0x0000017BFCD55790>, <__main__.Pod object at 0x0000017BFCD55970>, <__main__.Pod object at 0x0000017BFCD551F0>, <__main__.Pod object at 0x0000017BFCD55A30>, <__main__.Pod object at 0x0000017BFCD55D30>, <__main

In [27]:
season.valid_tops[0]

In [28]:
cache_team = matchups[season.valid_tops[0]][24][0]
print(cache_team)

[1, 2, 3]


In [29]:
matchups[cache_team][24]

In [93]:
def are_assignments_complete(assignments):
    return bool(
        min(assignments['top'][key] for key in assignments['top']) *
        min(assignments['flex'][key] for key in assignments['flex']) *
        min(assignments['bottom'][key] for key in assignments['bottom'])
    )

In [94]:
are_assignments_complete(assignments)

False

In [95]:
def are_pods_eligible_for_assignment(team, assignments):
    for pod in team.pods:
        if pod.is_top:
            if assignments['top'][pod] == 1:
                return False
        elif pod.is_bottom:
            if assignments['bottom'][pod] == 1:
                return False
        else:
            if assignments['flex'][pod] == 1:
                return False
    return True

In [96]:
def have_team_pods_played_together(team):
#     print(f'have_team_pod_played_together: {team} (type: {type(team)})')
    if team.pods[1] in team.pods[0].played_with or team.pods[2] in team.pods[0].played_with:
        return True
    return False

In [127]:
def get_pod_assignment(pod, assignments):
    if pod.is_top:
        return assignments['top'][pod]
    elif pod.is_bottom:
        return assignments['bottom'][pod]
    else:
        return assignments['flex'][pod]

In [134]:
# Get unique pods from tops: this will be used each round to determine which pods yet must be assigned
assignments = {
    'top': dict(),
    'bottom': dict(),
    'flex': dict()
}

for pod in season.sorted_pods:
    if pod.is_top:
        if pod not in assignments['top']:
            assignments['top'][pod] = 0
    elif pod.is_bottom:
        if pod not in assignments['bottom']:
            assignments['bottom'][pod] = 0
    else:
        if pod not in assignments['flex']:
            assignments['flex'][pod] = 0

In [135]:
schedule = dict()
iteration = 0

In [136]:
# Now to iterate through the season to assign actual matches based on the following constraints:
# 1. Never be in the same pod as another team more than once
# 2. Only play against opposing pods, at maximum, three times

cache_valid_tops = season.valid_tops.copy()
cache_valid_teams = season.valid_teams.copy()
cache_valid_bottoms = season.valid_bottoms.copy()

total_games = season.GAMES_PER_WEEK * season.SCHEDULE_WEEKS
MAX_PLAYED_AGAINST_COUNT = 2

eligibles = []


while len(cache_valid_teams) > 0:

    # 1. Pick a team
    team = cache_valid_teams.pop()
    print(f'Team: {team}')
    
    # 2. Validate these pods have never played with one-another
    # print(f'Pod {list(team.pods)[0]} has played with {list(team.pods)[0].played_with}')
    # print(f'Pod {list(team.pods)[1]} has played with {list(team.pods)[1].played_with}')
    # print(f'Pod {list(team.pods)[2]} has played with {list(team.pods)[2].played_with}')
    if have_team_pods_played_together(team):
        print(f'Pods have already played together... skipping and discarding team composition...')
        continue
    
    # 3. Iterate through eligible opponents and determine how many times they have played one-another
    deltas = list(matchups[team].keys())
    deltas.sort()
    print(f'deltas: {",".join([str(d) for d in deltas])}')
    for delta in deltas:
        opponents = matchups[team][delta]
        for opponent in opponents:
            # A. Make sure the opponents have not already played with one-another
            if not have_team_pods_played_together(opponent):
                # a. Check to make sure the pods haven't played against each other more than the maximum allowable
                max_played = 0
                for team_pod in team.pods:
                    for opp_pod in opponent.pods:
                        if team_pod.played_against.get(opp_pod, 0) > max_played:
                            max_played = team_pod.played_against.get(opp_pod, 0)
                if max_played <= MAX_PLAYED_AGAINST_COUNT:
#                     print(
#                         f'Appending eligible match: {team} vs. {opponent}. Delta: {delta}. '
#                         f'Max pod encounter: {max_played}.'
#                     )
                    eligibles.append((team, opponent, delta, max_played))


Team: [40, 41, 42]
deltas: 9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117
Team: [39, 41, 42]
deltas: 7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116
Team: [39, 40, 42]
deltas: 5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83

Team: [36, 39, 41]
deltas: 0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110
Team: [36, 39, 40]
deltas: 0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109
Team: [36, 38, 42]
deltas: 0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,8

Team: [35, 37, 39]
deltas: 0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105
Team: [35, 37, 38]
deltas: 0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104
Team: [35, 36, 42]
deltas: 0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,1

deltas: 0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105
Team: [34, 36, 40]
deltas: 0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104
Team: [34, 36, 39]
deltas: 0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103
Team: [3

Team: [33, 36, 37]
deltas: 0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100
Team: [33, 35, 42]
deltas: 0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104
Team: [33, 35, 41]
deltas: 0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103
Team: [33

...

Team: [1, 6, 29]
deltas: 0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87
Team: [1, 6, 28]
deltas: 0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88
Team: [1, 6, 27]
deltas: 0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89
Team: [1, 6, 26]
deltas: 0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,4

Team: [1, 5, 24]
deltas: 0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93
Team: [1, 5, 23]
deltas: 0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94
Team: [1, 5, 22]
deltas: 0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95
Team: [1, 5, 21]
deltas: 0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,2

Team: [1, 4, 20]
deltas: 0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98
Team: [1, 4, 19]
deltas: 0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99
Team: [1, 4, 18]
deltas: 0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100
Team: [1, 4, 17]
deltas: 0,1,2,3,4,5,6,7,8,9,10,11,12,

Team: [1, 3, 16]
deltas: 0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103
Team: [1, 3, 15]
deltas: 0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104
Team: [1, 3, 14]
deltas: 0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104

Team: [1, 2, 21]
deltas: 0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99
Team: [1, 2, 20]
deltas: 0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100
Team: [1, 2, 19]
deltas: 0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101
Team: [1, 2, 18]
deltas: 0,1,2,3,4,5,6,7,8,

In [137]:
len(eligibles)

69432904

In [ ]:
# Goal is to find a method of assigning all pods exclusively with the minimal delta possible:



def recursive_solving(eligibles, assignments, schedule=[]):
    
    # Terminating condition
    if len(eligibles) == 0:
        if not are_assignments_complete(assignments):
            print(f'Failure to allocate a complete schedule.')
        return schedule
    
    
    
    
    # Check for completeness
    if are_assignments_complete(assignments):
        return schedule
    # Run recursion if necessary
    else:
        match = eligibles[0]
        
        
    
    

In [138]:
eligibles[0]

(<__main__.Team at 0x236f635f820>, <__main__.Team at 0x236f635f100>, 9, 0)

In [139]:
eligibles[1]

(<__main__.Team at 0x236f635f820>, <__main__.Team at 0x236f635cd00>, 10, 0)

In [141]:
[str(pod) for pod in eligibles[0][0].pods], [str(pod) for pod in eligibles[0][1].pods]

(['40', '41', '42'], ['37', '38', '39'])

In [142]:
[str(pod) for pod in eligibles[1][0].pods], [str(pod) for pod in eligibles[1][1].pods]

(['40', '41', '42'], ['36', '38', '39'])

In [143]:
assignments

{'top': {<__main__.Pod at 0x236f60468e0>: 0,
  <__main__.Pod at 0x236f6046e20>: 0,
  <__main__.Pod at 0x236f6046e50>: 0,
  <__main__.Pod at 0x236f6046b20>: 0,
  <__main__.Pod at 0x236f60463a0>: 0,
  <__main__.Pod at 0x236f6046880>: 0,
  <__main__.Pod at 0x236f6089dc0>: 0,
  <__main__.Pod at 0x236f6089df0>: 0},
 'bottom': {<__main__.Pod at 0x236f6085df0>: 0,
  <__main__.Pod at 0x236f6085250>: 0,
  <__main__.Pod at 0x236f6085a90>: 0,
  <__main__.Pod at 0x236f60850d0>: 0,
  <__main__.Pod at 0x236f6085e80>: 0,
  <__main__.Pod at 0x236f6085e50>: 0,
  <__main__.Pod at 0x236f60858b0>: 0,
  <__main__.Pod at 0x236f6085b50>: 0},
 'flex': {<__main__.Pod at 0x236f6085070>: 0,
  <__main__.Pod at 0x236f6085280>: 0,
  <__main__.Pod at 0x236f6085370>: 0,
  <__main__.Pod at 0x236f60856d0>: 0,
  <__main__.Pod at 0x236f60852e0>: 0,
  <__main__.Pod at 0x236f6085520>: 0,
  <__main__.Pod at 0x236f60854c0>: 0,
  <__main__.Pod at 0x236f6085610>: 0,
  <__main__.Pod at 0x236f6085fa0>: 0,
  <__main__.Pod at 0x23

In [144]:
eligibles[0]    

(<__main__.Team at 0x236f635f820>, <__main__.Team at 0x236f635f100>, 9, 0)

In [145]:
eligibles[1]

(<__main__.Team at 0x236f635f820>, <__main__.Team at 0x236f635cd00>, 10, 0)

In [150]:
lineup = assignments['top'].copy()
lineup.update(assignments['bottom'].copy())
lineup.update(assignments['flex'].copy())
# lineup

In [ ]:
# direction -> [forward, reverse]


def solve(index, eligibles, assignments, team_stack=[], direction='forward', matches=[]):
    
    mask = eligibles[index:]
    team = mask[0]
    
    if direction == 'forward':
        if team == team_stack[-1]:
            # Already placed, move-on
            
            
        
        
    
    if team == team_stack[-1]:
        # Already placed this team
    

for event in eligibles:
    e1 = event[0]
    e2 = event[1]
    
    for pod in e1.pods:
        